In [9]:
import dotenv
dotenv.load_dotenv()

True

In [10]:
from datasets import load_dataset
swebench = load_dataset('princeton-nlp/SWE-bench', split='test')

# Randomly select a test set with 10 samples
test_set = swebench.shuffle(seed=42).select(range(10))

from prediction import do_prediction
results = []

for i in range(10):
    results.append(do_prediction(test_set[i]))
    
print(results)